# RANSAC: RAndom SAmple Concencus

An algorithm for model fitting with noisey data

It follows an iterative 3 step process.
1. Sample the minimum ammount of data to construct a model
2. Fit a model to that data
3. Check the remaining data to see if it fits the model

These steps are repeated untill a predetermined inlier threshold is reached.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from data.load_line import load_line
from utils.lines import Line2D

## 2D Line Fitting

Here, we are trying to perform 2D linear regression on noisy data. 

For a sample data, 10 points are sampled on a line, and 3 are uniformly sampled "outlier" points.

In [ ]:
line_points = load_line(noise_sigma = 0.05)
outlier_points = np.random.rand(2,3)

points = np.hstack((line_points, outlier_points))

plt.scatter(line_points[0,:], line_points[1,:], c='b')
plt.scatter(outlier_points[0,:], outlier_points[1,:], c='r')


### Step 1
Sample without replacement two points to fit line to

### Step 2
Fit a line to those points

internally the line is paramterize in slope-intercept form
$$ y = m x_p + b$$

we determine these parameters with two points $ a = [x_a, y_a], b = [x_b, y_b]$ using the following equations

$$m = \frac{y_b - y_a}{x_b-x_a}$$

$$b = y_a - m x_a$$

### Step 3
Check for inliers.

For determining the distance between a query point $q = [x_q, y_q]$ we use the following equation.

$$ \frac{| x_p - m * y_p - b | }{\sqrt{1 +b^2}}
$$


**The following cell is meant to be run iteratively.**

In [ ]:
# Step 1: Sample n=2 points.
random_indexs = np.random.choice(points.shape[1], size=2, replace=False)

# Step 2: Fit line to 2 points.
sampled_line = Line2D.FromTwoPoints(points[:,random_indexs[0]], points[:, random_indexs[1]])

# Step 3: Check Remaing point for inliers.
THRESHOLD = 0.1
inlier = []
for i in range(0,points.shape[1]):
    inlier.append(sampled_line.distance_to_point(points[:,i]) < THRESHOLD)
    
outlier = [not i for i in inlier]

# Plotting code.
plt.scatter(points[0, inlier], points[1, inlier], c='b')
plt.scatter(points[0, outlier], points[1, outlier], c='r')
plt.scatter(points[0, random_indexs], points[1, random_indexs], c='g')
plt.axline([0,sampled_line.b], slope=sampled_line.m)
    
print(sum(inlier))